<a href="https://colab.research.google.com/github/hyunaeee/KU-SW-Academy/blob/main/0915.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
HTTP response => Content-type: text/html
HTML 파싱? 1. RE
DOM(Document Object Model) => HTML Tags => Node => Tree
                              --------- <- ----
DOM 탐색; find_*(recursive=depth+-1/그 이상, limit=몇개)
         find(자식/자손), find_parent(부모/조상)
         ===> HTML 구조적 관계를 이용한 탐색 방법
CSSOM 탐색; Selector(태그, #id, .class, [key=value])
           => 가상선택자(Pseudo), :nth-child, :nth-of-type, ..., :has,:not
           => 관계+속성(find_*(attrs={}))
           => ( )로 자손, (>)로 자식
e.g. dom.html.body...., dom.body
     dom body True, dom > body False
     dom a#id or dom a.class or dom a.classA.classB; a태그 classA and classB
                                dom a.classA .classB; a태그 classA 자손 classB
     select, select_one
html = '''
<div id="result1">
<p class=row>
<a class="red" href="/link1">링크1</a>
<a class="blue" href="/link2">링크2</a>
</p>
<p class=row>
<a class="red" href="/link1">링크1</a>
<a class="blue" href="/link2">링크2</a>
</p>
</div>
<div id="result2">
<p class=row>
<a class="red" href="/link1">링크1</a>
<a class="blue" href="/link2">링크2</a>
</p>
<p class=row>
<a class="red" href="/link1">링크1</a>
<a class="blue" href="/link2">링크2</a>
</p>
</div>
'''
from bs4 import BeautifulSoup

dom = BeautifulSoup(html, 'html5lib')
# id=result1 자식 p의 자식들 중에 a
div = dom.find(attrs={'id':'result1'})
p = div.find(recursive=False)
p.find_all(recursive=False)
[<a class="red" href="/link1">링크1</a>, <a class="blue" href="/link2">링크2</a>]
div = dom.select_one('#result1')
p = div.select_one('p')
p.select('a')
[<a class="red" href="/link1">링크1</a>, <a class="blue" href="/link2">링크2</a>]
dom.select('#result1 > p > a')
# id=result1
# <p> 자식
#  <a></a>
#  <a></a>
# </p>
# <p> 자식
#  <a></a>
#  <a></a>
# </p>
[<a class="red" href="/link1">링크1</a>,
 <a class="blue" href="/link2">링크2</a>,
 <a class="red" href="/link1">링크1</a>,
 <a class="blue" href="/link2">링크2</a>]
dom.select('#result1 > p:first-child > a')
dom.select('#result1 > p:nth-child(1) > a')
[<a class="red" href="/link1">링크1</a>, <a class="blue" href="/link2">링크2</a>]
# +; next_sibling
# p + p(*)
# (*)p:has(+ p)
dom.select('#result1 > p:has(+ p) > a')
dom.select('#result1 > p + p > a')
[<a class="red" href="/link1">링크1</a>, <a class="blue" href="/link2">링크2</a>]
dom.select('#result1 > p:has(+ p) > a')[0]\
 is dom.select('#result1 > p + p > a')[0]
False
<div id="result1">
 <p id="p1"> <----
 </p>
 <p id="p2">
 </p>
</div>
<div id="result1">
 <p id="p1"> <---
 </p>
 <p> X
</div>
# id 위에처럼 중복 절대 X, id 대신 class라 생각
#result1 > p:has(+ p + p) => #p1
#result1 > p + p => #p2
dom.find('article').find_all('a')
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
Cell In [22], line 1
----> 1 dom.find('article').find_all('a')

AttributeError: 'NoneType' object has no attribute 'find_all'
dom.select('article a')
[]
from requests import request
from requests.exceptions import HTTPError
from time import sleep

def download(url, params={}, method='GET', retries=3):
    resp = None

    try:
        resp = request(method, url,
                       params=params if method=='GET' else {},
                       data=params if method=='POST' else {},
                       headers={'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'})
        resp.raise_for_status()
        # 아니면,
        # if resp.status_code != 200:
    except HTTPError as e:
        if 500 <= e.response.status_code:
            if retries > 0:
                sleep(3)
                resp = download(url, params=params,
                                method=method,
                                retries=retries-1)
            else:
                print('재방문 횟수 초과')
        else:
            print('Request', resp.request.headers)
            print('Response', e.response.headers)

    return resp
resp = download('http://pythonscraping.com/pages/page3.html')
dom = BeautifulSoup(resp.text, 'html5lib')
<table
 <tbody>
    <tr>
        <th>
            <img>
        </th>
        <td>
            <img>
 </tbody>
</table>
  Cell In [27], line 1
    <table>
    ^
SyntaxError: invalid syntax
table = dom.find('table')
tbody = table.find('tbody')
for tr in tbody.find_all('tr'):
    print(type(tr.find_all(recursive=False)[-1]))
    print(tr.find_all(recursive=False)[-1])
<class 'bs4.element.Tag'>
<th>
Image
</th>
<class 'bs4.element.Tag'>
<td>
<img src="../img/gifts/img1.jpg"/>
</td>
<class 'bs4.element.Tag'>
<td>
<img src="../img/gifts/img2.jpg"/>
</td>
<class 'bs4.element.Tag'>
<td>
<img src="../img/gifts/img3.jpg"/>
</td>
<class 'bs4.element.Tag'>
<td>
<img src="../img/gifts/img4.jpg"/>
</td>
<class 'bs4.element.Tag'>
<td>
<img src="../img/gifts/img6.jpg"/>
</td>
dom.select('table > tbody > tr > td > img')[0].find_parent()
<td>
<img src="../img/gifts/img1.jpg"/>
</td>
dom.select('tr:has(td:has( > img[src$=jpg]))')
[<tr class="gift" id="gift1"><td>
 Vegetable Basket
 </td><td>
 This vegetable basket is the perfect gift for your health conscious (or overweight) friends!
 <span class="excitingNote">Now with super-colorful bell peppers!</span>
 </td><td>
 $15.00
 </td><td>
 <img src="../img/gifts/img1.jpg"/>
 </td></tr>,
 <tr class="gift" id="gift2"><td>
 Russian Nesting Dolls
 </td><td>
 Hand-painted by trained monkeys, these exquisite dolls are priceless! And by "priceless," we mean "extremely expensive"! <span class="excitingNote">8 entire dolls per set! Octuple the presents!</span>
 </td><td>
 $10,000.52
 </td><td>
 <img src="../img/gifts/img2.jpg"/>
 </td></tr>,
 <tr class="gift" id="gift3"><td>
 Fish Painting
 </td><td>
 If something seems fishy about this painting, it's because it's a fish! <span class="excitingNote">Also hand-painted by trained monkeys!</span>
 </td><td>
 $10,005.00
 </td><td>
 <img src="../img/gifts/img3.jpg"/>
 </td></tr>,
 <tr class="gift" id="gift4"><td>
 Dead Parrot
 </td><td>
 This is an ex-parrot! <span class="excitingNote">Or maybe he's only resting?</span>
 </td><td>
 $0.50
 </td><td>
 <img src="../img/gifts/img4.jpg"/>
 </td></tr>,
 <tr class="gift" id="gift5"><td>
 Mystery Box
 </td><td>
 If you love suprises, this mystery box is for you! Do not place on light-colored surfaces. May cause oil staining. <span class="excitingNote">Keep your friends guessing!</span>
 </td><td>
 $1.50
 </td><td>
 <img src="../img/gifts/img6.jpg"/>
 </td></tr>]
dom.select('tbody > tr:nth-child(n)')
dom.select('tbody > tr:nth-child(even|2n)')
dom.select('tbody > tr:nth-child(odd|2n-1)')
[<tr><th>
 Item Title
 </th><th>
 Description
 </th><th>
 Cost
 </th><th>
 Image
 </th></tr>,
 <tr class="gift" id="gift2"><td>
 Russian Nesting Dolls
 </td><td>
 Hand-painted by trained monkeys, these exquisite dolls are priceless! And by "priceless," we mean "extremely expensive"! <span class="excitingNote">8 entire dolls per set! Octuple the presents!</span>
 </td><td>
 $10,000.52
 </td><td>
 <img src="../img/gifts/img2.jpg"/>
 </td></tr>,
 <tr class="gift" id="gift4"><td>
 Dead Parrot
 </td><td>
 This is an ex-parrot! <span class="excitingNote">Or maybe he's only resting?</span>
 </td><td>
 $0.50
 </td><td>
 <img src="../img/gifts/img4.jpg"/>
 </td></tr>]
resp = download('https://www.google.com/search', {'q':'카리나'})
resp.status_code, resp.headers['content-type']
(200, 'text/html; charset=UTF-8')
dom = BeautifulSoup(resp.text, 'html5lib')
import re
dom.find(text=re.compile('나무위키')).find_parent().find_parent().find_parent().find_parent()
<a data-ved="2ahUKEwjNjLKKsquBAxWqZ_EDHZGkD2AQFnoECE0QAQ" href="https://namu.wiki/w/%EC%B9%B4%EB%A6%AC%EB%82%98(aespa)" jsaction="rcuQ6b:npT2md" jscontroller="M9mgyc" jsname="UWckNb" ping="/url?sa=t&amp;source=web&amp;rct=j&amp;opi=89978449&amp;url=https://namu.wiki/w/%25EC%25B9%25B4%25EB%25A6%25AC%25EB%2582%2598(aespa)&amp;ved=2ahUKEwjNjLKKsquBAxWqZ_EDHZGkD2AQFnoECE0QAQ"><br/><h3 class="LC20lb MBeuO DKV0Md">카리나(aespa)</h3><div class="notranslate TbwUpd NJjxre iUh30 ojE3Fb"><span class="H9lube"><div aria-hidden="true" class="eqA2re NjwKYd Vwoesf"><img alt="" class="XNo5Ab" src="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAACAAAAAgCAMAAABEpIrGAAAAwFBMVEUBp5oCp5oDp5kFqJcGqJYHqZQKqpAOq40QrIsRrIoTrYgUrYcVroYXroMYr4MZr4Ebr4AcsH8dsH4esXsfsXohsnkisncjs3Yks3Ums3QntHMAn4YApYV6yLvp9vP0/Pue1spdvq/////b8OmR0cK34doApGdewKfS7OQAqns7uI4+tp3E5t4ArnOo3MWX1L4ArmuDzLhPvJpawJkAn2614cwAoXoytJVyx6uM0bau3NWl2sfE5+FCtaZPu6Rkv7KC2ANCAAABE0lEQVR4AdyRU4IDQQBEY9vJ9Oj1mLFx/1OtrQPsf7kymWwuXygWS6VSuVyp1uqNZqvd6fb6g+FoPJnOZv8LoIjynwBV0w3xB8BEk1iiUq03mz8BijaKo0lRdT3P9X9SMDADPOECuO2fMuhAKCKUGM9/AsRxPPrYwpK6EBGlPtETIE68dP5ph5CFCAGWjxaxRCeafwQIqYlqaGv6Y8jeCmu9YfxpSZWwWt3u2Lfbfnz4ASCItjsb5Lqb6vDZQlGPprDw4JRiQ7R6CJl8CHkGbB1t0Wj4cGoNPtcsXDg2PaLrtlarN0/4X4cq3LitE0Tl6awGB/8roAge8vXNfev71KHUo/vB0wPWuJCWZsORYEZIxgEAuatI9jt5CO8AAAAASUVORK5CYII=" style="height:18px;width:18px"/></div></span><div><span class="VuuXrf">나무위키</span><div class="byrV5b"><cite class="qLRx3b tjvcx GvPZzd cHaqb" role="text">https://namu.wiki<span class="dyjrff ob9lvb" role="text"> › 카리나(aespa)</span></cite></div></div></div></a>
dom.select('h3')

# H3
len(dom.select('.LC20lb.MBeuO.DKV0Md'))
len(dom.select('.LC20lb'))
len(dom.select('h3.LC20lb'))
len(dom.select('h3.LC20lb.MBeuO.DKV0Md'))

len(dom.select('a[jsname="UWckNb"] > h3'))
len(dom.select('a > h3.LC20lb'))
len(dom.select('a > h3.LC20lb.MBeuO.DKV0Md'))

# A(자식으로 H3를 갖고 있는)
len(dom.select('a[jsname="UWckNb"]:has( > h3)'))
len(dom.select('a:has( > h3.LC20lb)'))
len(dom.select('a:has( > h3.LC20lb.MBeuO.DKV0Md)'))
7
for a in dom.select('a:has( > h3.LC20lb.MBeuO.DKV0Md)'):
    print(a.attrs['href'])
    print(a.select_one('h3').text)
https://namu.wiki/w/%EC%B9%B4%EB%A6%AC%EB%82%98(aespa)
카리나(aespa)
https://www.instagram.com/karina_aespas_/
AESPA KARINA 카리나 (@karina_aespas_)
https://ko.wikipedia.org/wiki/%EC%B9%B4%EB%A6%AC%EB%82%98_(%EA%B0%80%EC%88%98)
카리나 (가수) - 위키백과, 우리 모두의 백과사전
https://www.youtube.com/shorts/Dul1Q0SK0n4
Chicago #pinkchampagne #aespa #æspa #에스파 ... - YouTube
https://namu.wiki/w/%EC%B9%B4%EB%A6%AC%EB%82%98(aespa)?rev=3170
카리나(aespa) (r3170 판)
https://www.youtube.com/watch?v=v63rTR68TdA
230625 aespa(에스파) 'Next Level' 카리나 KARINA 4K Cam ...
https://www.youtube.com/watch?v=gO_ONOnzH-s
#PictureChallenge with #KARINA #GISELLE #HYO #효연 ...
resp = download('https://search.daum.net/search', {'w':'tot',
                                                   'q':'카리나'})
resp.status_code, resp.headers['content-type']
(200, 'text/html; charset=utf-8')
dom = BeautifulSoup(resp.text, 'html5lib')
# 뉴스의 링크
dom.select('a.tit_main.fn_tit_u') # 속성만 이용(클래스)
dom.select('.wrap_cont > a.tit_main') # 부모 관계와 속성 이용
dom.select('.wrap_thumb[id] + .wrap_cont > a')
for a in dom.select('#container > li > div:last-child > a'):
    print(a.text.strip())
    print(a.attrs['href'])
에스파 카리나, 걸어 다니는 인형… 일상이 화보
https://v.daum.net/v/20230903232408019?f=o
카리나, 중단발 사진 공개…단발 욕구↑
https://v.daum.net/v/20230911172827349?f=o
에스파 카리나, 모자 커 보이는 착시 효과… ‘소두 인증’
https://v.daum.net/v/20230911185502680?f=o
카리나, 모자 커 보이는 착시효과..뉴욕시 '천사 강림' [스타IN★]
https://v.daum.net/v/20230911183359250?f=o
for title in dom.select('c-doc-web > c-title'):
    print(title.text)
    print(title.attrs['data-href'])
카리나 (배우)
https://ko.wikipedia.org/wiki/%EC%B9%B4%EB%A6%AC%EB%82%98%20%28%EB%B0%B0%EC%9A%B0%29
카리나(가디언 테일즈) - 나무위키
https://namu.wiki/w/%EC%B9%B4%EB%A6%AC%EB%82%98(%EA%B0%80%EB%94%94%EC%96%B8%20%ED%85%8C%EC%9D%BC%EC%A6%88)
데뷔 이래 가장 짧은 머리인 카리나
https://cafe.daum.net/subdued20club/ReHf/4523290?q=%EC%B9%B4%EB%A6%AC%EB%82%98&re=1
카리나 달글 8 100일동안 작성한📝 카푸🐱의 성장🌱일기를 볼 때는 👀 주위는 환하게💡 최대한 오래 🩵💙
https://cafe.daum.net/Duckgu/D2nu/41411?q=%EC%B9%B4%EB%A6%AC%EB%82%98&re=1
고윤정 vs 카리나
https://cafe.daum.net/ssaumjil/LnOm/3029160?q=%EC%B9%B4%EB%A6%AC%EB%82%98&re=1
손흥민이 월드컵 우승시키기 vs 카리나랑 결혼하기
https://cafe.daum.net/dotax/Elgq/4240303?q=%EC%B9%B4%EB%A6%AC%EB%82%98&re=1
AI로 만든 에스파 카리나 바비.jpg
https://table.cafe.daum.net/p/1217111531/207167064921750272
에스파 윈터 가뿐하게 공주님 안기 성공하는 지젤 보고 호기롭게 도전하는 카리나
https://table.cafe.daum.net/p/1000055110/176847531157393664
최근 화제가 되고 있는 에스파 카리나 다이어트와 다이어트 식단
https://nizniz.tistory.com/149
시에나를 제대로 즐기는 방법 - 만지아의 탑, 캄포광장,시에나 대성당, 카타리나 그리고 카리나
https://brunch.co.kr/@gle-bay/204
원샷 카리나 - 카카오스토리
http://story.kakao.com/_cOgo99/jKqXPaREfBa
for a in dom.select('''
#container > li > div:last-child > a,
c-doc-web > c-title
'''):
    print(a.text.strip())
    print(a.attrs[
        ('' if a.has_attr('href') else 'data-') + 'href'])
에스파 카리나, 걸어 다니는 인형… 일상이 화보
https://v.daum.net/v/20230903232408019?f=o
카리나, 중단발 사진 공개…단발 욕구↑
https://v.daum.net/v/20230911172827349?f=o
에스파 카리나, 모자 커 보이는 착시 효과… ‘소두 인증’
https://v.daum.net/v/20230911185502680?f=o
카리나, 모자 커 보이는 착시효과..뉴욕시 '천사 강림' [스타IN★]
https://v.daum.net/v/20230911183359250?f=o
카리나 (배우)
https://ko.wikipedia.org/wiki/%EC%B9%B4%EB%A6%AC%EB%82%98%20%28%EB%B0%B0%EC%9A%B0%29
카리나(가디언 테일즈) - 나무위키
https://namu.wiki/w/%EC%B9%B4%EB%A6%AC%EB%82%98(%EA%B0%80%EB%94%94%EC%96%B8%20%ED%85%8C%EC%9D%BC%EC%A6%88)
데뷔 이래 가장 짧은 머리인 카리나
https://cafe.daum.net/subdued20club/ReHf/4523290?q=%EC%B9%B4%EB%A6%AC%EB%82%98&re=1
카리나 달글 8 100일동안 작성한📝 카푸🐱의 성장🌱일기를 볼 때는 👀 주위는 환하게💡 최대한 오래 🩵💙
https://cafe.daum.net/Duckgu/D2nu/41411?q=%EC%B9%B4%EB%A6%AC%EB%82%98&re=1
고윤정 vs 카리나
https://cafe.daum.net/ssaumjil/LnOm/3029160?q=%EC%B9%B4%EB%A6%AC%EB%82%98&re=1
손흥민이 월드컵 우승시키기 vs 카리나랑 결혼하기
https://cafe.daum.net/dotax/Elgq/4240303?q=%EC%B9%B4%EB%A6%AC%EB%82%98&re=1
AI로 만든 에스파 카리나 바비.jpg
https://table.cafe.daum.net/p/1217111531/207167064921750272
에스파 윈터 가뿐하게 공주님 안기 성공하는 지젤 보고 호기롭게 도전하는 카리나
https://table.cafe.daum.net/p/1000055110/176847531157393664
최근 화제가 되고 있는 에스파 카리나 다이어트와 다이어트 식단
https://nizniz.tistory.com/149
시에나를 제대로 즐기는 방법 - 만지아의 탑, 캄포광장,시에나 대성당, 카타리나 그리고 카리나
https://brunch.co.kr/@gle-bay/204
원샷 카리나 - 카카오스토리
http://story.kakao.com/_cOgo99/jKqXPaREfBa
CSS Selector; 구조+속성
Crawler = Crawling + Scraping(나중에)
          => URL
             => HyperLink = A[href/data-href/****]
                            IMG[src], IFRAME[src], FROM[action]
URLs = []
seed = URLs 주소를 하나씩 꺼내서

while URLs: # 종료 조건; 더이상 URL이 없을때까지
    seed => /(루트)에 가서 robots.txt
    seed => (HTTP) Req/Resp
    Resp.status_code, Resp.Headers
    content-type:text/html
    HTML => DOM
    온갖 링크 추출
    링크를 절대주소의 형태로 변환
    링크를 방문한적이 있는가?
    URLs.append(새롭게 찾은 링크)
AI: Search Space(탐색 -> Tree/Graph -> DFS, BFS, + Heuristic;Focused)
ML: Data-driven Modeling
DL: NN-based, Cost(Loss/Error) => Convex
A:100, [A-1:200, A-1-1:, A-2:.. 500, 10, ...]
                  Root:정보를 담고 있는 페이지
         Node               Node
Node         Node     Node         Node...
Wiki
BFS => Queue(FIFO/LILO)
DFS => Stack(FILO/LIFO)
     ================
H => A, B, C, D <= T
     ================
pop, push => 자료구조 => 동적 => C
from requests.compat import urljoin, urlparse

url = 'https://www.google.com/search?q=%EC%B9%B4%EB%A6%AC%EB%82%98&oq=%EC%B9%B4%EB%A6%AC%EB%82%98&gs_lcrp=EgZjaHJvbWUqBggAEEUYOzIGCAAQRRg7Mg0IARAAGIMBGLEDGIAEMg0IAhAAGIMBGLEDGIAEMg0IAxAAGIMBGLEDGIAEMg0IBBAAGIMBGLEDGIAEMhMIBRAuGIMBGMcBGLEDGNEDGIAEMgYIBhBFGDwyBggHEEUYPNIBBzY2NWowajeoAgCwAgA&sourceid=chrome&ie=UTF-8'

URLs = []
URLs.append((url, 0))
# (url, depth) [0]:url, [1]:depth

seens = []
domain = ['www.google.com']

# 전략; BFS(Queue)/DFS(Stack)
#    ; Focused Crawling(depth, domain, content, ...)
# 1. Depth
# 2. Domain

while URLs: # 종료 조건; 더이상 URL이 없을때까지
    seed = URLs.pop(-1)
    seens.append(seed[0])
    # robots.txt 확인

    resp = download(seed[0])

    if resp.status_code != 200:
        continue

    if re.search('text/html', resp.headers['content-type']):
        dom = BeautifulSoup(resp.text, 'html5lib')
        for link in dom.select('*[href], *[src], *[action]'):
            if link.has_attr('src'):
                href = link.attrs['src']
            elif link.has_attr('href'):
                href = link.attrs['href']
            elif link.has_attr('action'):
                href = link.attrs['action']

            if not re.match('(?:#)|(?:javascript)|(?:data)', href):
                # 링크를 절대주소의 형태로 변환
                newurl = urljoin(resp.request.url, href)
                # 링크를 방문할예정이거나 방문한적이 있는가?

                # depth 제한
                if seed[1] > 3:
                    continue

                # whitelist; opt-in
                # blacklist; opt-out; in으로 바꾸면
                if urlparse(newurl).netloc not in domain:
                    continue

                if newurl not in list(map(lambda r:r[0], seens)) and\
                   newurl not in list(map(lambda r:r[0], URLs)):
                    # URLs.append(새롭게 찾은 링크)
                    URLs.append((newurl, seed[1]+1))
---------------------------------------------------------------------------
KeyboardInterrupt                         Traceback (most recent call last)
Cell In [126], line 22
     19 seens.append(seed[0])
     20 # robots.txt 확인
---> 22 resp = download(seed[0])
     24 if resp.status_code != 200:
     25     continue

Cell In [24], line 9, in download(url, params, method, retries)
      6 resp = None
      8 try:
----> 9     resp = request(method, url,
     10                    params=params if method=='GET' else {},
     11                    data=params if method=='POST' else {},
     12                    headers={'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'})
     13     resp.raise_for_status()
     14     # 아니면,
     15     # if resp.status_code != 200:

File /opt/homebrew/anaconda3/lib/python3.9/site-packages/requests/api.py:59, in request(method, url, **kwargs)
     55 # By using the 'with' statement we are sure the session is closed, thus we
     56 # avoid leaving sockets open which can trigger a ResourceWarning in some
     57 # cases, and look like a memory leak in others.
     58 with sessions.Session() as session:
---> 59     return session.request(method=method, url=url, **kwargs)

File /opt/homebrew/anaconda3/lib/python3.9/site-packages/requests/sessions.py:587, in Session.request(self, method, url, params, data, headers, cookies, files, auth, timeout, allow_redirects, proxies, hooks, stream, verify, cert, json)
    582 send_kwargs = {
    583     "timeout": timeout,
    584     "allow_redirects": allow_redirects,
    585 }
    586 send_kwargs.update(settings)
--> 587 resp = self.send(prep, **send_kwargs)
    589 return resp

File /opt/homebrew/anaconda3/lib/python3.9/site-packages/requests/sessions.py:701, in Session.send(self, request, **kwargs)
    698 start = preferred_clock()
    700 # Send the request
--> 701 r = adapter.send(request, **kwargs)
    703 # Total elapsed time of the request (approximately)
    704 elapsed = preferred_clock() - start

File /opt/homebrew/anaconda3/lib/python3.9/site-packages/requests/adapters.py:489, in HTTPAdapter.send(self, request, stream, timeout, verify, cert, proxies)
    487 try:
    488     if not chunked:
--> 489         resp = conn.urlopen(
    490             method=request.method,
    491             url=url,
    492             body=request.body,
    493             headers=request.headers,
    494             redirect=False,
    495             assert_same_host=False,
    496             preload_content=False,
    497             decode_content=False,
    498             retries=self.max_retries,
    499             timeout=timeout,
    500         )
    502     # Send the request.
    503     else:
    504         if hasattr(conn, "proxy_pool"):

File /opt/homebrew/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:703, in HTTPConnectionPool.urlopen(self, method, url, body, headers, retries, redirect, assert_same_host, timeout, pool_timeout, release_conn, chunked, body_pos, **response_kw)
    700     self._prepare_proxy(conn)
    702 # Make the request on the httplib connection object.
--> 703 httplib_response = self._make_request(
    704     conn,
    705     method,
    706     url,
    707     timeout=timeout_obj,
    708     body=body,
    709     headers=headers,
    710     chunked=chunked,
    711 )
    713 # If we're going to release the connection in ``finally:``, then
    714 # the response doesn't need to know about the connection. Otherwise
    715 # it will also try to release it and we'll have a double-release
    716 # mess.
    717 response_conn = conn if not release_conn else None

File /opt/homebrew/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:449, in HTTPConnectionPool._make_request(self, conn, method, url, timeout, chunked, **httplib_request_kw)
    444             httplib_response = conn.getresponse()
    445         except BaseException as e:
    446             # Remove the TypeError from the exception chain in
    447             # Python 3 (including for exceptions like SystemExit).
    448             # Otherwise it looks like a bug in the code.
--> 449             six.raise_from(e, None)
    450 except (SocketTimeout, BaseSSLError, SocketError) as e:
    451     self._raise_timeout(err=e, url=url, timeout_value=read_timeout)

File <string>:3, in raise_from(value, from_value)

File /opt/homebrew/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:444, in HTTPConnectionPool._make_request(self, conn, method, url, timeout, chunked, **httplib_request_kw)
    441 except TypeError:
    442     # Python 3
    443     try:
--> 444         httplib_response = conn.getresponse()
    445     except BaseException as e:
    446         # Remove the TypeError from the exception chain in
    447         # Python 3 (including for exceptions like SystemExit).
    448         # Otherwise it looks like a bug in the code.
    449         six.raise_from(e, None)

File /opt/homebrew/anaconda3/lib/python3.9/http/client.py:1377, in HTTPConnection.getresponse(self)
   1375 try:
   1376     try:
-> 1377         response.begin()
   1378     except ConnectionError:
   1379         self.close()

File /opt/homebrew/anaconda3/lib/python3.9/http/client.py:320, in HTTPResponse.begin(self)
    318 # read until we get a non-100 response
    319 while True:
--> 320     version, status, reason = self._read_status()
    321     if status != CONTINUE:
    322         break

File /opt/homebrew/anaconda3/lib/python3.9/http/client.py:281, in HTTPResponse._read_status(self)
    280 def _read_status(self):
--> 281     line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
    282     if len(line) > _MAXLINE:
    283         raise LineTooLong("status line")

File /opt/homebrew/anaconda3/lib/python3.9/socket.py:704, in SocketIO.readinto(self, b)
    702 while True:
    703     try:
--> 704         return self._sock.recv_into(b)
    705     except timeout:
    706         self._timeout_occurred = True

File /opt/homebrew/anaconda3/lib/python3.9/ssl.py:1242, in SSLSocket.recv_into(self, buffer, nbytes, flags)
   1238     if flags != 0:
   1239         raise ValueError(
   1240           "non-zero flags not allowed in calls to recv_into() on %s" %
   1241           self.__class__)
-> 1242     return self.read(nbytes, buffer)
   1243 else:
   1244     return super().recv_into(buffer, nbytes, flags)

File /opt/homebrew/anaconda3/lib/python3.9/ssl.py:1100, in SSLSocket.read(self, len, buffer)
   1098 try:
   1099     if buffer is not None:
-> 1100         return self._sslobj.read(len, buffer)
   1101     else:
   1102         return self._sslobj.read(len)

KeyboardInterrupt:
len(URLs), len(seens)
(1399, 41)
list(map(lambda r:urlparse(r[0]).netloc, URLs))
https://www.google.com/search?q=&oq=&gs_lcrp=&sourceid=&ie=UTF-8
https://www.google.com/search?q=&oq=&gs_lcrp=&sourceid=&
https://www.google.com/search?q=다른값&oq=&gs_lcrp=&sourceid=&
    http or https
    www.google.com; netloc; host; domain *
    /search [dis]allow Y/N? 시점?